# DMRG-Like Solution for Advection-Diffusion Equation with Periodic Boundary Conditions, With Fit Algorithm

In [ ]:
using HDF5
using ITensors
using LinearAlgebra
using Plots
using ITensorMPS
using Observers
using LaTeXStrings
plot_font = "Computer Modern";

### Construction of the initial condition as MPS

In [ ]:
N = 4;  # Number of sites
s = siteinds("Qubit", N);   # Indices for all sites
Re = 32;    # Reynolds Number
dt = 0.001; # Time step (Cosider the stability criteria: Δt ≤ (Re/2)Δx²)
Time = 1; # Final simulation time
numsteps = Int.(round(Time/dt)); # Number of time steps

χ = 20; # Maximum truncation parameter
ϵ = 1e-10; # Truncation error

### Initial condition

In [ ]:
xs = range(0, 1-1/(2^N), length=2^N); # Sampling grid points

ys = [0.0 for i in 1:2^N] # Initialization

# Box, equal to 1 in the central half of the system

for j = 1:2^N    
    if (xs[j] >= 0.25) && (xs[j] <= 0.75)
        ys[j] = 1.0;
    end
end

plot(xs,ys)

In [ ]:
# Create MPS from the function
ψ0 = MPS(ys, s, cutoff=ϵ, maxdim=χ);

orthogonalize!(ψ0,1)

@show norm(ψ0)

### Construction of the derivative operations as MPO

In [ ]:
step_size = 1/2^N

### Construction of dudx and d2udx2 using 8th order central finite difference

# Left shift:

#     Input
#
#   |   |   |
#   O - O - O
#   |   |   |
#
#     Output

left_ls = zeros(2, 2, 2) #(D, U, R)
middle_ls = zeros(2, 2, 2, 2) #(L, D, U, R)
right_ls = zeros(2, 2, 2) #(L, D, U)

left_ls[1, 2, 2] = 1
left_ls[2, 1, 1] = 1
middle_ls[1, 1, 1, 1] = 1
middle_ls[1, 2, 2, 1] = 1
middle_ls[2, 2, 1, 1] = 1
middle_ls[2, 1, 2, 2] = 1
right_ls[1, 1, 1] = 1
right_ls[1, 2, 2] = 1
right_ls[2, 1, 2] = 1
right_ls[2, 2, 1] = 1

H_ls = MPO(N)

α = [Index(2) for i in 1:(N-1)] # bond indices

for i = 1:N
    if i == 1 # first site
        H_ls[i] = ITensor(left_ls, s[i], s[i]', α[i])
    elseif i == N # last site
        H_ls[i] = ITensor(right_ls, α[i-1], s[i], s[i]')
    else # middle site
        H_ls[i] = ITensor(middle_ls, α[i-1], s[i], s[i]', α[i])
    end
end

orthogonalize!(H_ls,1)

# Right shift:

#     Input
#
#   |   |   |
#   O - O - O
#   |   |   |
#
#     Output

left_rs = zeros(2, 2, 2) #(D, U, R)
middle_rs = zeros(2, 2, 2, 2) #(L, D, U, R)
right_rs = zeros(2, 2, 2) #(L, D, U)

left_rs[1, 2, 1] = 1
left_rs[2, 1, 2] = 1
middle_rs[1, 1, 1, 1] = 1
middle_rs[1, 2, 2, 1] = 1
middle_rs[2, 1, 2, 1] = 1
middle_rs[2, 2, 1, 2] = 1
right_rs[1, 1, 1] = 1
right_rs[1, 2, 2] = 1
right_rs[2, 1, 2] = 1
right_rs[2, 2, 1] = 1

H_rs = MPO(N)

α = [Index(2) for i in 1:(N-1)] # bond indices

for i = 1:N
    if i == 1 # first site
        H_rs[i] = ITensor(left_rs, s[i], s[i]', α[i])
    elseif i == N # last site
        H_rs[i] = ITensor(right_rs, α[i-1], s[i], s[i]')
    else # middle site
        H_rs[i] = ITensor(middle_rs, α[i-1], s[i], s[i]', α[i])
    end
end

orthogonalize!(H_rs,1)

# Identity:

#     Input
#
#   |   |   |
#   O - O - O
#   |   |   |
#
#     Output

left_id = zeros(2, 2, 1) #(D, U, R)
middle_id = zeros(1, 2, 2, 1) #(L, D, U, R)
right_id = zeros(1, 2, 2) #(L, D, U)

left_id[1, 1, 1] = 1
left_id[2, 2, 1] = 1
middle_id[1, 1, 1, 1] = 1
middle_id[1, 2, 2, 1] = 1
right_id[1, 1, 1] = 1
right_id[1, 2, 2] = 1

H_id = MPO(N)

α = [Index(1) for i in 1:(N-1)] # bond indices

for i = 1:N
    if i == 1 # first site
        H_id[i] = ITensor(left_id, s[i], s[i]', α[i])
    elseif i == N # last site
        H_id[i] = ITensor(right_id, α[i-1], s[i], s[i]')
    else # middle site
        H_id[i] = ITensor(middle_id, α[i-1], s[i], s[i]', α[i])
    end
end

orthogonalize!(H_id,1)

# Define derivatives
H_dudx = 0.5*(H_ls - H_rs)/step_size; # First derivative
H_d2udx2 = (H_rs + H_ls - 2*H_id)/(step_size^2); # Second derivative

### DMRG-Like Solver

In [ ]:
function Advection_Diffusion_Fit(ψ0, H_dudx, H_d2udx2, H_id, Re, s, N, dt, numsteps, χ, ϵ)
    
    ψ_list = []
    ψ = ψ0
    nsweeps = 5
    init = randomMPS(s)

    # Perfor time evolution
    for t = 1:numsteps
    
        # Define full MPO to advance one step in time
        H = (dt/Re)*H_d2udx2 - dt*H_dudx + H_id;    
        
        # Advance state by one time step using Fit algorithm
        ψ = apply(H, ψ, alg="fit", init=init, maxdim=χ, cutoff=ϵ, nsweeps=nsweeps, outputlevel=0, normalize=false) 

        if t % (numsteps/4) == 0
        
            Big_ψ = contract(ψ) # Recovering the original big tensor
            J_recon = Array(Big_ψ, s) # Changing the format from ITensor to Julia array for plotting
            ys_recon = reshape(J_recon,2^N,1) # Reshaping from N-dimentional tensor to regular array
        
            push!(ψ_list, ys_recon);
        end

    end

    return ψ_list, maxlinkdim(ψ);

end

In [ ]:
ψ_list, max_χ = Advection_Diffusion_Fit(ψ0, H_dudx, H_d2udx2, H_id, Re, s, N, dt, numsteps, χ, ϵ);

In [ ]:
plot(xs, ys, labels="t = 0", lw=3, linestyle=:solid, palette=:okabe_ito, framestyle=:box, dpi=1000, gridalpha = 0, legend=false)
plot!(xs, ψ_list[1], labels="t = 0.25", lw=3, linestyle=:dash, xguidefontsize=15, yguidefontsize=15,legendfontsize=10)
plot!(xs, ψ_list[2], labels="t = 0.5", lw=3, linestyle=:dot)
plot!(xs, ψ_list[3], labels="t = 0.75", lw=3, linestyle=:dashdot, color = 6)
plot!(xs, ψ_list[4], labels="t = 1", lw=3, linestyle=:dashdotdot)

### Simulation finished!!!